## 추천시스템(Recommendation System) - 2

### 02 아이유팬이 좋아할 만한 아티스트?

#### 02-01 추천 시스템이란?
- Q1) 협업 필터링(Collaborative Filtering) 방식과 콘텐츠 기반 필터링(Contents-based Filtering) 방식의 차이
  - 협업 필터링 : 다른 사용자 이력 기반으로, 사용자간 유사성 찾기
  - 콘텐츠 기반 : 콘텐츠(상품) 자체의 유사성 찾기
- Q2) 협업 필터링을 바로 사용할 수 없게 만드는 세가지 제약 조건?
  - Cold start문제
  - 연산량이 많이 필요
  - 좋아하는 아이템(편중되는 장르)만 추천하는 문제
    - 특정 정치성향에 쏠린 영상만 보는 소비 패턴을 떠올리면 이해가 빠를 듯
- Q3) 유튜브 뮤직의 첫화면에서 처음 접속한 사용자에게 좋아하는 아티스트 5명 이상의 정보를 요구하는 이유?
  - Cold start문제 극복하려고?


#### 02-02 데이터 탐색하기와 전처리
- (리눅스 명령어) more ~/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv
- 굳이 vi를 사용해서 1.6Gb 짜리 데이터 전체를 통째로 메모리 상에 열어보지는 않기
  - 궁금해서 컨테이너 내의 메모리 확인해봄
  - 리눅스 CLI에서는 free -h 명령어 사용하면 됨(GPU포함 자원 알고 싶으면 top 사용)
  - 근데 GPU사용 안하는 모듈에서는 nvidia -smi로 CUDA가 잡혀있어도, GPU자체가 할당되진 않음.
    - Devops, MLops 관점에서 매우 흥미로운 부분

##### 데이터 불러오기

In [2]:
# 1.6GB 짜리 직접 열어보지 말라는 얘기가 있어서, 메모리 확인을 해봤다
import psutil
ram = psutil.virtual_memory()
print(f"Total: {ram.total / (1024**3):.2f} GB")
print(f"Available: {ram.available / (1024**3):.2f} GB")
print(f"Used: {ram.used / (1024**3):.2f} GB")

Total: 17.56 GB
Available: 16.61 GB
Used: 0.67 GB


In [3]:
import pandas as pd
import os

fname = os.getenv('HOME') + '/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 임의로 지정한 컬럼명
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있습니다.  
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


In [4]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [5]:
data['artist'] = data['artist'].str.lower() # 검색을 쉽게 하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [6]:
condition = (data['user_id']== data.loc[0, 'user_id'])
data.loc[condition]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


##### 데이터 탐색

In [7]:
# 유저 수
data['user_id'].nunique()

358868

In [8]:
# 아티스트 수
data['artist'].nunique()

291346

In [9]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [10]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [11]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

##### 모델 검증을 위한 사용자 초기 정보 세팅
- 넷플릭스, 유투브 등은 최초에 사용자에게 좋아하는 장르, 음악 등 5개 이상 물어봄
- Cold start문제 해결하기 위한 목적

In [12]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['black eyed peas' , 'maroon5' ,'jason mraz' ,'coldplay' ,'beyoncé']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = pd.concat([data, my_playlist])                            # 위에 임의로 만든 my_playlist 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,black eyed peas,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,beyoncé,30


##### 모델에 활용하기 위한 전처리 (실습)
- 인덱싱 : user와 artist 각각에 번호 붙이는 것

In [13]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [14]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zimin'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(artist_to_idx['black eyed peas'])

358868
376


In [15]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing OK!!
artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,376,30
1,358868,270115,30
2,358868,3746,30
3,358868,62,30


#### 02-03 사용자 평가
- 사용자가 별점(명시적)을 준 것이 아니라, 얼마나 찾았느냐도 평가 지표로 활용 가능(암묵적)
- [참고논문 Collaborative Filtering for Implicit Feedback Datasets](http://yifanhu.net/PUB/cf.pdf)
- "암묵적 피드백 데이터셋"의 문제
  - 별점처럼 명시적 평가가 아니고, '부정적 피드백(별 0개, 1개)'이 없음
  - 잡음이 많음
  - 수치는 신뢰도를 의미
  - Implicit-feedback Recommender System은 적절한 평가가 필요
- 여기서는 편의상
  - 한 번 이상 감상했으면 -> 긍정적인 피드백
  - 음악이 많이 재생된 아티스트에 대해 가중치

In [16]:
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['play']==1]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

147739,17535660
Ratio of only_one over all data is 0.84%


#### 02-04 Matrix Factorization(MF)
- m명의 사용자들이 n명의 아티스트에 대해 평가한 데이터(행렬)을 만든다.
- MF 모델은 2006년 Netflix에서 백만 달러의 상금을 걸고 개최한 자사 추천 시스템의 성능을 10% 이상 향상시키는    
  챌린지를 계기로 알려지게 됨

#### 02-05 CSR행렬
- 유저는 36만 명이고 아티스트는 29만 명, 각 cell마다 1byte씩(정수1개) 들어간다 가정하면,
  36만 * 29만 * 1byte ~ 약 97GB
- 그런데 '모두의 연구소'에서 각 사용자 컨테이너(pod)별로 분배된 메모리(RAM) 크기는 16GB 내외
  - free -h (RAM확인), top(GPU포함 확인)
  - 우린 '학습 노드'별로 환경이 바뀌는 컨테이너 구조여서, GPU 안 쓰는 경우면 top 입력해도 GPU는 안보임
  - nvidia -smi치면 CUDA 잡히는 것이 보이는 건 흥미로움
- 결론적으로 CSR(Compressed Sparse Row) 행렬을 사용
  - CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성

In [20]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<358869x291347 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Row format>

#### 02-06 MF 모델 학습
- Matrix Factorization 모델을 implicit 패키지로 학습
- implicit 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습
- 이 패키지에 구현된 als(AlternatingLeastSquares) 모델을 사용
- AlternatingLeastSquares 클래스의 __init__ 파라미터를 살펴보겠습니다.
  - factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
  - regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
  - use_gpu : GPU를 사용할 것인지
  - iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

In [21]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [22]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [23]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<291347x358869 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Column format>

In [24]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [25]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

print('슝=3')

슝=3


In [26]:
zimin_vector

array([ 0.08768566,  1.0262684 ,  0.10907838,  0.04045577,  0.10405017,
        0.12452897, -0.2556262 ,  1.2551593 ,  0.0220086 ,  0.5845711 ,
       -0.12375359, -1.536488  , -0.93317986, -0.8560482 , -0.04570324,
        0.64189696, -0.12529503,  0.0666386 , -0.9569837 ,  0.21389183,
        0.20183994,  0.98887193,  0.11748716,  0.6135178 ,  0.41651252,
       -1.129039  , -0.19566324, -0.11622605, -0.86814684,  0.8012663 ,
       -0.58201116,  1.4137037 , -0.8684223 ,  0.7121489 ,  0.47016293,
       -0.22905836, -0.52658796, -0.02779153, -0.10250244,  0.4029236 ,
       -0.11831156,  0.19648895, -1.5271171 ,  0.23827429,  0.17390138,
       -0.65211743,  0.48618275,  0.3730032 ,  0.4119072 , -0.04608208,
        0.49357268,  0.5104842 ,  1.4779989 , -0.15860574, -0.45961717,
       -0.04019175,  0.73960125,  0.7293353 ,  1.5476757 ,  0.20331094,
       -0.43421665, -0.3008234 , -0.5844656 ,  0.03541479, -0.2496629 ,
        1.0345464 , -1.401744  , -1.1648844 , -1.036242  ,  0.87

In [27]:
black_eyed_peas_vector

array([ 0.01499332,  0.01969675,  0.02881931,  0.00482177,  0.02040482,
        0.01149229, -0.01433684,  0.0187917 ,  0.00741214,  0.01860149,
        0.00216891, -0.01510548, -0.00278057, -0.00072432, -0.00837808,
        0.01516884,  0.00477174,  0.00032868, -0.0060069 ,  0.01141248,
        0.01244965,  0.01769893,  0.00349795,  0.00899725,  0.00627046,
       -0.00757306,  0.01408418,  0.00398955, -0.02147617,  0.00899704,
        0.01157629,  0.02430557, -0.0123477 ,  0.00130404,  0.01661584,
        0.00266282,  0.01598836,  0.00052492, -0.00657459,  0.02102337,
       -0.00282577, -0.00552835,  0.00492195,  0.02702221,  0.00672341,
       -0.01442553,  0.02320656,  0.01456359,  0.00234888, -0.00444935,
        0.01257089,  0.01138974,  0.01127084,  0.00799567, -0.00957732,
        0.00422201,  0.02015569,  0.00382794,  0.03288581,  0.01949971,
        0.00891916,  0.00608164, -0.0067556 ,  0.01801432,  0.00817555,
        0.01272856, -0.00572782,  0.00263508, -0.0043484 ,  0.01

In [28]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(zimin_vector, black_eyed_peas_vector)

0.5013544

In [29]:
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

0.2896722